In [40]:
import os
import cv2
import numpy as np
base_dir = "D:/6610210437/03-10/dataset"
def load_images_from_folder(folder):
    images = []
    for filename in os. listdir(folder) :
        img = cv2. imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images
base_dir = "D:/6610210437/03-10/dataset"
apple_images = load_images_from_folder(os.path.join(base_dir, "apple"))
orange_images = load_images_from_folder(os.path.join(base_dir, "orange"))

print (f"Loaded {len(apple_images) } apple images")
print (f"Loaded {len(orange_images) } orange images")


Loaded 50 apple images
Loaded 49 orange images


In [41]:
#3. ปรับ ขนาด รูปภาพ ให้ เท่า กัน
# You might want to resize or preprocess these images later
# Example: resize to a common size (e.g., 100r100)
resized_apple_images = [cv2.resize(img, (100, 100)) for img in apple_images]
resized_orange_images = [cv2.resize(img, (100, 100)) for img in orange_images]

In [42]:
#4. แปลง รูปภาพ เป็น feature vector
# Flatten the resized images into feature vectora
apple_features = [img.flatten() for img in resized_apple_images]
orange_features = [img.flatten() for img in resized_orange_images]
print(f"Flattened apple features shape (first image): {apple_features[0].
shape}")
print(f"Flattened orange featuree shape (first image): {orange_features[0].
shape}")

Flattened apple features shape (first image): (30000,)
Flattened orange featuree shape (first image): (30000,)


In [43]:
#5. สร้าง โมเดล
from sklearn.model_selection import train_test_split

# Create labela: 0 for apple, 1 for orange
apple_labels = [0] * len(apple_features)
orange_labels = [1] * len(orange_features)

# Combine features and labels
all_features = apple_features + orange_features
all_labels = apple_labels + orange_labels

# Convert to numpy arrays
X = np.array(all_features)
y = np.array(all_labels)

#Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training features shape: {X_train.shape}")
print(f"Testing features shape: {X_test.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Testing labels shape: {y_test.shape}")


Training features shape: (79, 30000)
Testing features shape: (20, 30000)
Training labels shape: (79,)
Testing labels shape: (20,)


In [44]:
#6.ประเมินประสิทธิภาพ
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train the SVM classifier
svm_model = svm.SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.90

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.92      0.92        12
           1       0.88      0.88      0.88         8

    accuracy                           0.90        20
   macro avg       0.90      0.90      0.90        20
weighted avg       0.90      0.90      0.90        20



In [45]:
#7.บันทึกโมเดล
import joblib
#Save the trained model to a file
filename = 'svm_image_classifier_model.pkl'
joblib.dump(svm_model, filename)

print(f"Model save to {filename}")

Model save to svm_image_classifier_model.pkl


In [46]:
%%writefile app.py
import streamlit as st
import joblib
import cv2  # ✅ สำคัญ! ต้อง import
from PIL import Image
import numpy as np

# --- Load the trained model ---
with open("svm_image_classifier_model.pkl", "rb") as f:
    model = joblib.load(f)

# --- Streamlit UI ---
st.title("Fruit Classifier")
st.write("Upload an image of an **apple** or **orange**, and the model will predict it.")

# --- File uploader ---
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg"])

# --- Define class labels ---
class_dict = {0: "Apple", 1: "Orange"}

if uploaded_file is not None:
    # Display image
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption='Uploaded Image', use_container_width=True)

    if st.button("Predict"):
        # ✅ Resize image to 100x100 (เหมือนตอนฝึกโมเดล)
        image = image.resize((100, 100))

        # ✅ Convert to numpy array
        image_array = np.array(image)

        # ✅ Convert RGB to BGR (ตามลำดับสีของ OpenCV)
        image_array = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)

        # ✅ Flatten และ reshape เป็น input ที่โมเดลต้องการ
        image_array = image_array.flatten().reshape(1, -1)

        # ✅ Predict
        prediction = model.predict(image_array)[0]
        prediction_name = class_dict[prediction]

        # ✅ Show result
        st.markdown(f"### Prediction: **{prediction_name}**")


Overwriting app.py
